In [1]:
import os
import pickle
import sys
import math
sys.path.append("../")
from od.env.config import SUMO_SIM_SECONDS
from od.misc.statistic import NetFlowType
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.env.config import SUMO_SIM_STEPS,ROOT_DIR
from od.env.station import BS_PRESET
import od.misc.statistic as oms

In [2]:
print(os.getcwd())
interest_config = InterestConfig(True,ResourceAllocatorType.NOMA_APR,False,0.6,666)
statistic = None
print(interest_config)
with open("../data/TVAL/{}/object.pickle".format(interest_config.folder()), "rb") as file:
    statistic = pickle.load(file)
statistic.nft_traffic = {}

/home/OD/repos/ProjectSumo/ipynb
yQoS-nRSU-NOMA_APR-D0.6-S666


In [110]:

BS_CONFIG =  {
        name:bs_config 
        for name,bs_config in BS_PRESET.items()
        if (
            bs_config["type"] == BaseStationType.UMA  or 
            (
                interest_config.req_rsu and 
                bs_config["type"] == BaseStationType.UMI
            )
        )
}

class FakeBaseStation:
    def __init__(self,name,pos,bs_type,serial):
        self.name = name
        self.pos = pos
        self.type = bs_type
        self.serial = serial
    def __repr__(self) -> str:
        return self.name
    def __index__(self):
        return self.serial
        
class GV:
    NET_STATION_CONTROLLER = [ 
        FakeBaseStation(
            name,
            BS_CONFIG[name]["pos"],
            BS_CONFIG[name]["type"] ,
            i
        )   for i,name in enumerate(BS_CONFIG)
    ]

class STATISTIC:
    def Log(data):
        print(data)

In [111]:
GV.NET_STATION_CONTROLLER

[UMA-1, UMA-2]

In [146]:
TARGET_NFT = NetFlowType.GENERAL

In [148]:
def VehicleAppdataArrivalRateReport(self, app_stats):
        time_orgnz_record = {}
        _max = 0
        _min = 0
        _avg = 0
        _num = 0
        if(len(app_stats) > 0):
            for record in app_stats.values():
                time = record.at
                if(time not in time_orgnz_record):
                    time_orgnz_record[time] = 1
                else:
                    time_orgnz_record[time] += 1
            _max = max(time_orgnz_record.values())
            _min = min(time_orgnz_record.values())
            _avg = sum(time_orgnz_record.values())/(self.RecordDuration()*1000)
            _num = sum(time_orgnz_record.values())
        # print(time_orgnz_record)
        return {
            "max":_max,
            "avg":_avg,
            "min":_min,
            "num":_num
        }
VehicleAppdataArrivalRateReport(statistic,statistic.ExtractNetworkTraffic(TARGET_NFT,False))

{'max': 21, 'avg': 8.2305, 'min': 1, 'num': 16461}

In [149]:
def VehicleAppdataArrivalSizeReport(self, app_stats):
        time_orgnz_record = {}
        _max = 0
        _min = 0
        _avg = 0
        _num = 0
        if(len(app_stats) > 0):
            for record in app_stats.values():
                time = record.at
                if(time not in time_orgnz_record):
                    time_orgnz_record[time] =[]
                time_orgnz_record[time].append(record.bits)
            per_ms_avg_data_size = [ sum(x)/len(x) for x in time_orgnz_record.values() ]
            all_data_size = [ v for x in time_orgnz_record.values() for v in x]
            _max = max(per_ms_avg_data_size)
            _min = min(per_ms_avg_data_size)
            _avg = sum(all_data_size)/len(all_data_size)
            _num = len(all_data_size)
        return {
            "max":_max,
            "avg":_avg,
            "min":_min,
            "num":_num
        }
VehicleAppdataArrivalSizeReport(statistic,statistic.ExtractNetworkTraffic(TARGET_NFT,False))

{'max': 14508.0, 'avg': 8528.871878986696, 'min': 2188.0, 'num': 16461}

In [150]:
def VehicleAppdataDepartRateReport(self, app_stats):
        time_orgnz_record = {}
        _max = 0
        _min = 0
        _avg = 0
        _num = 0
        if(len(app_stats) > 0):
            for record in app_stats.values():
                time_enter_bs_txq = [ 
                    time_per_bs_txq[0][0]
                    for time_per_bs_txq in record.time_bs_txq 
                    if len(time_per_bs_txq) > 0
                ]
                if(len(time_enter_bs_txq) > 0):
                    time = min(time_enter_bs_txq)
                    if(time not in time_orgnz_record):
                        time_orgnz_record[time] = 1
                    else:
                        time_orgnz_record[time] += 1
            _max = max(time_orgnz_record.values())
            _min = min(time_orgnz_record.values())
            _avg = sum(time_orgnz_record.values())/(self.RecordDuration()*1000)
            _num = sum(time_orgnz_record.values())
        # print(time_orgnz_record)
        return {
            "max":_max,
            "avg":_avg,
            "min":_min,
            "num":_num
        }
VehicleAppdataDepartRateReport(statistic,statistic.ExtractNetworkTraffic(TARGET_NFT,False))

{'max': 15, 'avg': 7.3885, 'min': 1, 'num': 14777}

In [151]:
def BaseStationAppdataArrivalRateReport(self, app_stats):
        bs_report = []
        if(len(app_stats) > 0 ):
            for bs in GV.NET_STATION_CONTROLLER:
                time_orgnz_record = {}
                _max = 0
                _min = 0
                _avg = 0
                _num = 0
                for record in app_stats.values():
                    if(len(record.time_bs_txq[bs]) > 0):
                        time = record.time_bs_txq[bs][0][0]
                        if(not time in  time_orgnz_record):
                            time_orgnz_record[time] = 1
                        else:
                            time_orgnz_record[time] += 1
                if(len(time_orgnz_record) > 0):
                    _max = max(time_orgnz_record.values())
                    _min = min(time_orgnz_record.values())
                    _avg = sum(time_orgnz_record.values())/self.RecordDuration()/1000
                    _num = sum(time_orgnz_record.values())
                bs_report.append(
                    {
                        "max": _max,
                        "avg": _avg,
                        "min": _min,
                        "num": _num
                    }
                )
        else:
            bs_report = [ 
                {
                        "max": 0,
                        "avg": 0,
                        "min": 0,
                        "num": 0
                }
                for bs in GV.NET_STATION_CONTROLLER
            ]

        return bs_report

BaseStationAppdataArrivalRateReport(statistic,statistic.ExtractNetworkTraffic(TARGET_NFT,False))

[{'max': 15, 'avg': 7.3885, 'min': 1, 'num': 14777},
 {'max': 15, 'avg': 7.3885, 'min': 1, 'num': 14777}]

In [152]:
def BaseStationAppdataArrivalSizeReport(self, app_stats):
        bs_report = []
        if(len(app_stats) > 0 ):
            for bs in GV.NET_STATION_CONTROLLER:
                time_orgnz_record = {}
                _max = 0
                _min = 0
                _avg = 0
                _num = 0
                for record in app_stats.values():
                    if(len(record.time_bs_txq[bs]) > 0):
                        time = record.time_bs_txq[bs][0][0]
                        if(not time in  time_orgnz_record):
                            time_orgnz_record[time] = []
                        time_orgnz_record[time].append(record.bits)
                if(len(time_orgnz_record) > 0):
                    per_ms_avg_data_size = [ sum(x)/len(x) for x in time_orgnz_record.values()]
                    total_data_size = [x for i in time_orgnz_record.values() for x in i]
                    _max = max(per_ms_avg_data_size)
                    _min = min(per_ms_avg_data_size)
                    _avg = sum(total_data_size)/len(total_data_size)
                    _num = len(total_data_size)
                bs_report.append(
                    {
                        "max": _max,
                        "avg": _avg,
                        "min": _min,
                        "num": _num
                    }
                )
        else:
            bs_report = [ 
                {
                        "max": 0,
                        "avg": 0,
                        "min": 0,
                        "num": 0
                }
                for bs in GV.NET_STATION_CONTROLLER
            ]

        return bs_report

BaseStationAppdataArrivalSizeReport(statistic,statistic.ExtractNetworkTraffic(TARGET_NFT,False))

[{'max': 16384.0, 'avg': 8517.795492995872, 'min': 552.0, 'num': 14777},
 {'max': 16384.0, 'avg': 8517.795492995872, 'min': 552.0, 'num': 14777}]

In [153]:
def BaseStationAppdataDepartRateReport(self, app_stats):
        bs_report = []
        if(len(app_stats) > 0 ):
            for bs in GV.NET_STATION_CONTROLLER:
                time_orgnz_record = {}
                _max = 0
                _min = 0
                _avg = 0
                _num = 0
                for record in app_stats.values():
                    time = record.time_bs_serv[bs]
                    if(time > 0):
                        if(not time in time_orgnz_record):
                            time_orgnz_record[time] = 1
                        else:
                            time_orgnz_record[time] += 1
                if(len(time_orgnz_record) > 0):
                    _max = max(time_orgnz_record.values())
                    _min = min(time_orgnz_record.values())
                    _avg = sum(time_orgnz_record.values())/self.RecordDuration()/1000
                    _num = sum(time_orgnz_record.values())
                bs_report.append(
                    {
                        "max": _max,
                        "avg": _avg,
                        "min": _min,
                        "num": _num
                    }
                )
        else:
            bs_report = [ 
                {
                        "max": 0,
                        "avg": 0,
                        "min": 0,
                        "num": 0
                }
                for _ in GV.NET_STATION_CONTROLLER
            ]

        return bs_report

BaseStationAppdataDepartRateReport(statistic,statistic.ExtractNetworkTraffic(TARGET_NFT,False))

[{'max': 15, 'avg': 7.2515, 'min': 1, 'num': 14503},
 {'max': 14, 'avg': 7.3885, 'min': 1, 'num': 14777}]

In [154]:
def BaseStationUplinkConnectionReport(self, app_stats):
        bs_report = []
        if(len(app_stats) > 0 ):
            for bs in GV.NET_STATION_CONTROLLER:
                time_orgnz_record = {}
                _max = 0
                _min = 0
                _avg = 0
                _num = 0
                for name,record in app_stats.items():
                    # print(name)
                    if(len(record.time_bs_txq[bs]) > 0):
                        time = record.time_bs_txq[bs][0][0]
                        if(not time in  time_orgnz_record):
                            time_orgnz_record[time] = set()
                        time_orgnz_record[time].update(name[:name.find('-')])
                per_ms_uplink_conn = [len(x) for x in time_orgnz_record.values()]
                if(len(time_orgnz_record) > 0):
                    _max = max(per_ms_uplink_conn)
                    _min = min(per_ms_uplink_conn)
                    _avg = sum(per_ms_uplink_conn)/self.RecordDuration()/1000
                    _num = sum(per_ms_uplink_conn)
                bs_report.append(
                    {
                        "max": _max,
                        "avg": _avg,
                        "min": _min,
                        "num": _num
                    }
                )
        else:
            bs_report = [ 
                {
                        "max": 0,
                        "avg": 0,
                        "min": 0,
                        "num": 0
                }
                for bs in GV.NET_STATION_CONTROLLER
            ]

        return bs_report

BaseStationUplinkConnectionReport(statistic,statistic.ExtractNetworkTraffic(TARGET_NFT,False))

[{'max': 16, 'avg': 20.01, 'min': 8, 'num': 40020},
 {'max': 16, 'avg': 20.01, 'min': 8, 'num': 40020}]